In [3]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPSsePlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [22]:
from semantic_kernel import Kernel
from user_plugins import WeatherPlugin, DataCenterPlugin
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio

from semantic_kernel.connectors.mcp import MCPSsePlugin

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



## Connecting to a remote MCP server via sse

- Server-Sent Events (SSE) is a mechanism for establishing a continuous, one-way data stream from the server to a client, such as a web application. This allows the server to push updates or messages to the client without the client needing to constantly poll for new information

In [23]:
async with MCPSsePlugin(
    name="weather",
    url=f"{environ["MCP_SERVER_URL"]}/weather/sse",
) as weather_plugin:

    agent = ChatCompletionAgent(
        kernel=kernel, 
        name="WeatherAgent", 
        plugins=[weather_plugin, ]
    )
    response = await agent.get_response(messages="What's the weather like in sf today?")
    print(response)

It's 60°F and foggy in San Francisco today.


In [ ]:
from contextlib import AsyncExitStack

plugin_names = ["weather", "search", "energy"]

messages = [
    "What data centers are in 'critical'?",
    "Describe the resource intensity of data center facility infrastructure"
]


async with AsyncExitStack() as stack:
    plugins = []
    for plugin_name in plugin_names:
        plugin = await stack.enter_async_context(
            MCPSsePlugin(
                name=plugin_name,
                url=f"{environ['MCP_SERVER_URL']}/{plugin_name}/sse"
            )
        )
        plugins.append(plugin)

    agent = ChatCompletionAgent(
        kernel=kernel,
        name="MultiPluginAgent",
        plugins=plugins
    )


    for msg in messages:
        print("------------------------------------------------\n")
        print(msg)
        response = await agent.get_response(messages=msg)
        print(response)
        print("\n------------------------------------------------\n")


------------------------------------------------

What data centers are in 'critical'?
The following data centers are currently marked as "critical":

1. **DC-NYC1**, Zone: B2
   - Timestamp: 2025-04-14T15:00:13
   - Power Draw: 190.82 kW
   - PUE (Power Usage Effectiveness): 1.29
   - Temperature: 29.66°C
   - Notes: No operator notes provided.

2. **DC-CHI2**, Zone: A1
   - Timestamp: 2025-04-14T14:00:13
   - Power Draw: 162.3 kW
   - PUE: 2.05
   - Temperature: 24.69°C
   - Notes: "Temp spike investigated."

3. **DC-NYC1**, Zone: B2
   - Timestamp: 2025-04-14T13:45:13
   - Power Draw: 163.71 kW
   - PUE: 1.72
   - Temperature: 25.68°C
   - Notes: "Checked UPS levels."

4. **DC-CHI2**, Zone: C3
   - Timestamp: 2025-04-14T13:15:13
   - Power Draw: 224.52 kW
   - PUE: 1.58
   - Temperature: 25.54°C
   - Notes: "Rebooted cooling system."

5. **DC-OMA1**, Zone: A1
   - Timestamp: 2025-04-14T12:30:13
   - Power Draw: 122.23 kW
   - PUE: 1.78
   - Temperature: 29.32°C
   - Notes: "Temp spi